In [ ]:
# import necessary packages
import numpy as np
import pandas as pd

# path to data we just downloaded
# if you are interested in how modelpoints were created,
# check out the `generating_modelpoints.ipynb` file in the repo
DATA_XLS = "./dstoolkit/model_point.xlsx"

# ingesting our data into a dataframe (df)

### Creating & editting spreadsheets from CLI with `openpyxl` & 'xlwings`

In [2]:
%pip install openpyxl xlwings

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 3.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 12.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for lxml: filename=lxml-4.9.2-cp310-cp310-macosx_11_0_arm64.whl size=1605713 sha256=6149d4d7ba26c498d7e2c8448db69912028b00d2e684a5d7564fa4518e2a4429
  Stored in directory: /Users/neelanpather/Library/Caches/pip/wheels/5a/51/0e/95b4a6ddee4a616530c36aeb03dafb5e04183756d9973a7d5d
Successfully built lxml
Note: you may need to restart the kernel to use updated packages.


In [3]:
import xlwings as xw
import pandas as pd

# Interactive mode: open new workbook
# write pd.DataFrame to "A1" then read it back

book = xw.Book()
sheet = book.sheets[0]
sheet.range("A1").value = pd.DataFrame({"A": [1, 2, 3], "B": [4, 5, 6]})
df = sheet.range("A1").options(pd.DataFrame, expand="table").value


XlwingsError: Make sure to have "appscript" and "psutil", dependencies of xlwings, installed.